In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
'''
This is a trail to test whether it is approachable to do surface modeling in single brain region.
'''
import time as t0
import matplotlib
from tvb.simulator.lab import *
import pandas as pd
from datetime import date
import dfa
import os
import numpy as np
tic = t0.time()

In [3]:
simulation_mode = 'simu1'
data_folder = os.path.join(os.getcwd(), 'data')

In [4]:
def plot_result(time, signal, interval, t_unit='ms'):
    #Plot region averaged time series
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(111)
    if signal.ndim == 4:
        ax.plot(time[interval[0]:interval[-1]], signal[interval[0]:interval[-1], 0, :, 0])
    elif signal.ndim == 1:
        ax.plot(time[interval[0]:interval[-1]], signal[interval[0]:interval[-1]])
    else:
        print("Dimension should be 4 or 1!")
        return None
    title("Region average")
    plt.xlabel("time/"+t_unit)
    #Show them
    show()

In [5]:
simulator.Simulator()
# specify the coupling
c=coupling.Sigmoidal(a=np.array([0.5]))
# region-based modeling
white_matter = connectivity.Connectivity.from_file()
tract_lengths = white_matter.tract_lengths[73:74,73:74]
region_labels = white_matter.region_labels[73:74]
centres = white_matter.centres[73:74]
cortical = white_matter.cortical[73:74]
orientations = white_matter.orientations[73:74]
areas = white_matter.areas[73:74]
weights = white_matter.weights[73:74,73:74]
one_region = connectivity.Connectivity(number_of_regions = 1,
                                        # number_of_connections=4,
                                        tract_lengths = tract_lengths,
                                        region_labels = region_labels,
                                        centres = centres,
                                        cortical = cortical,
                                        orientations = orientations,
                                        areas = areas,weights = weights)
one_region.configure()

WARNING  File 'hemispheres' not found in ZIP.


In [6]:
# The algorithm to solve differential equations
nois = noise.Multiplicative(ntau=0.5, nsig = np.array([0.05]), b=equations.Gaussian())  # ? Do we need to optimize ntau?
nois.configure_coloured(dt=0.1, shape=1)  # ? shape?? 
integ = integrators.HeunStochastic(dt=0.5, noise=nois)

In [7]:
# Initialize the model
c_ee = 11
c_ei = 14
# also test 15, 14 -> biggest delta in matlab:0.59, but here 0.53210353; 11, 14: 0.56463684
# smallest: 6,8
c_ie = 9.23
c_ii = 2.0
mod = models.WilsonCowan(k_e = np.array([1.0]),            #Max value of excitatory response function
                         k_i = np.array([2.0]),            #Max value of inhibitory response function
                         r_e = np.array([1.0]),            #Excitatory refractory period
                         r_i = np.array([1.0]),            #Inhibitory refractory period
                         c_ee = np.array([c_ee]),          #Excitatory to excitatory coupling coefficient
                         c_ei = np.array([c_ei]),          #Inhibitory to excitatory coupling coefficient
                         c_ie = np.array([c_ie]),         #Excitatory to inhibitory coupling coefficient
                         c_ii = np.array([c_ii]),          #Inhibitory to inhibitory coupling coefficient
                         tau_e = np.array([10.0]),         #Membrane time-constant for the excitatory population
                         tau_i = np.array([10.0]),         #Membrane time-constant for the inhibitory population
                         a_e = np.array([1.3]),            #Slope parameter for the excitatory response function
                         b_e = np.array([2.8]),           #Position of the maximum slope of the excitatory sigmoid function
                         c_e = np.array([7.0]),            #Amplitude parameter for the excitatory response function
                         a_i = np.array([2.0]),            #Slope parameter for the inhibitory response function
                         b_i = np.array([4.0]),            #Position of the maximum slope of the inhibitory sigmoid function
                         c_i = np.array([1.0]),            #Amplitude parameter for the inhibitory response function
                         theta_e = np.array([2.0]),        #Excitatory treshold
                         theta_i = np.array([1.5]),        #Inhibitory treshold
                         alpha_e = np.array([1.0]),        # ?Balance parameter between excitatory and inhibitory masses
                         alpha_i = np.array([1.0]),        # ?Balance parameter between excitatory and inhibitory masses
                         P = np.array([2.5]),              #External stimulus to the excitatory population
                         Q = np.array([0.0]),              #External stimulus to the inhibitory population
                         shift_sigmoid = np.array([False])
                        )

In [8]:
source_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/V1_180.zip'
mapping_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/regionMapping_180_1.txt'
local_connectivity_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/local_connectivity_180.mat'

In [14]:
#Initialise a surface
surf = surfaces.CorticalSurface.from_file(source_file=source_path)
surf.configure()

default_cortex = cortex.Cortex.from_file(source_file=source_path, region_mapping_file=mapping_path)
default_cortex.coupling_strength = np.array([2**-10])
default_cortex.local_connectivity = local_connectivity.LocalConnectivity.from_file(local_connectivity_path)
default_cortex.region_mapping_data.connectivity = one_region
default_cortex.region_mapping_data.surface = surf
default_cortex.configure()

In [15]:
if stimulation:
    # Define a stimulus
    eqn_t = equations.Alpha()  # belongs to a family of exponential function, used for visual stimulus # TODO Literature search!
    eqn_t.parameters['onset'] = 100  # ! Must specify the params, otherwise no value will present in temporal_pattern
    eqn_t.parameters['alpha'] = 13
    eqn_t.parameters['beta'] = 42

    eqn_s = equations.DiscreteEquation()
    stimulus = patterns.StimuliSurface(surface=default_cortex.surface,
                                       focal_points_triangles=np.array([154]),
                                       temporal=eqn_t,
                                       spatial = eqn_s)
    stimulus.configure_space()
    stimulus.configure_time(np.arange(0., 1000, 2**-1))  # apply a stimulus at 25s
    stimulus.configure_space()
    stimulus.configure_time(np.array([25000]))  # apply a stimulus at 25s
    plot_pattern(stimulus)

In [ ]:
#Initialise some Monitors with period in physical time
mon_savg = monitors.SpatialAverage(period=2**-1)
# load the default region mapping
# rm = region_mapping.RegionMapping.from_file(source_file=mapping_path)
# mon_eeg = monitors.EEG.from_file() # require whole brain mapping
# mon_eeg.region_mapping=rm
#Initialise Simulator -- Model, Connectivity, Integrator, Monitors, and surface.
sim = simulator.Simulator(model = mod, connectivity = one_region,
                          coupling = c, 
                          integrator = integ, monitors = (mon_savg,),
                          simulation_length = 1000000,
                          surface = default_cortex)

sim.configure()

In [ ]:
# sim.integrator.noise.nsig = np.array([1.]) #0.00001]) # configure the noise otherwise ERROR: Bad Simulator.integrator.noise.nsig shape

In [ ]:
#Perform the simulation
# tavg_data = []
# tavg_time = []
savg_data = []
savg_time = []
# eeg_data = []
# eeg_time = []
# hf = h5py.File('data.h5','w')
# hf.create_dataset('dts_1', data=d1)
today = date.today().isoformat()
simu_name = today + '_' + simulation_mode+'_'+str(c_ee)+'_'+str(c_ei)+'_'+str(c_ie)+'_'+str(c_ii)
result_name = os.path.join(data_folder,simu_name+"_results.csv")
batch = 0
last_t = []
latt_d = []
check_point = 50000
for savg in sim():
    # if not tavg is None:
        # tavg_time.append(tavg[0])
        # tavg_data.append(tavg[1])
    if not savg is None:
        savg_time.append(savg[0][0])
        savg_data.append(savg[0][1])
        if len(savg_time) == check_point:
            SAVG = np.array(savg_data)
            v1 = SAVG[:,0,0,0]
            # v2 = SAVG[:,0,1,0]
            df = pd.DataFrame({'time':savg_time,
                               's_v1':v1})
            if batch == 0:
                df.to_csv(result_name, index=False, header=False)
            elif batch == 1:
                if sum(abs(v1[-10000:])) < 200:
                    print("Oscillation is not generated. Will abandon this trail.")
                    plot_result(savg_time,v1,[40000,49999])
                    break
                else:
                    df.to_csv(result_name, mode='a', index=False, header=False)
            else:
                df.to_csv(result_name, mode='a', index=False, header=False)
            batch +=1
            last_t = savg_time
            last_d = savg_data
            df= []
            savg_data = []
            savg_time = []

    # if not eeg is None:
    #     eeg_time.append(eeg[0])
    #     eeg_data.append(eeg[1])

In [ ]:
# import tvb.datatypes.time_series
# tsr = tvb.datatypes.time_series.TimeSeriesRegion()
# SAVG = numpy.array(savg)
# tsr.data = SAVG
# tsr.sample_period = 2**-1
# import tvb.simulator.power_spectra_interactive as ps_int
# psi = ps_int.PowerSpectraInteractive(time_series=tsr)
# psi.show()

In [ ]:
# result = sim.run()

In [ ]:
# savg_time = numpy.array(last_t)
# savg_time = savg_time.astype(float)
# time_s = savg_time/10000

In [ ]:
if len(last_t) != 0:
    savg_time = np.array(last_t)
    time_s = savg_time/1000
    SAVG = np.array(last_d)
else:
    savg_time = np.array(savg_time)
    time_s = savg_time/1000
    SAVG = np.array(savg_data)

In [ ]:
# plot_result(time_s, SAVG, [5000,15000])
plot_result(time_s, SAVG, [0,2000],'s')

In [ ]:
# #Plot region averaged time series
# fig = plt.figure(figsize=(15,5))
# ax = fig.add_subplot(111)
# ax.plot(savg_time[5000:15000], SAVG[5000:15000, 0, :, 0])
# # ax.plot(time_s, SAVG[:,0,0,0])
# title("Region average")
# plt.xlabel("time/ms")
# show()

In [ ]:
toc = t0.time() - tic
print('Elapsed:',toc)

In [ ]:
df = pd.read_csv(result_name, header = None)
data = df[1].to_numpy()
raw = dfa.load_data([data],add_noise=True, noise_sd=0.01)
R , _ = dfa.compute_DFA(raw, l_freq=1, h_freq=4,method='richard')
R

In [ ]:
R_delta = R
R_delta

In [ ]:
# df = pd.read_csv(result_name, header = None)
# total_rows=len(df.axes[0])
# print("Number of Rows: "+str(total_rows))
# df.tail()